In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/final-20-22e/translator/fingerprint.pb
/kaggle/input/final-20-22e/translator/saved_model.pb
/kaggle/input/final-20-22e/translator/variables/variables.index
/kaggle/input/final-20-22e/translator/variables/variables.data-00000-of-00001
/kaggle/input/file-name-20-02/file_name_20_0.2/translator/fingerprint.pb
/kaggle/input/file-name-20-02/file_name_20_0.2/translator/saved_model.pb
/kaggle/input/file-name-20-02/file_name_20_0.2/translator/variables/variables.index
/kaggle/input/file-name-20-02/file_name_20_0.2/translator/variables/variables.data-00000-of-00001
/kaggle/input/translator-2/translator1/fingerprint.pb
/kaggle/input/translator-2/translator1/saved_model.pb
/kaggle/input/translator-2/translator1/variables/variables.index
/kaggle/input/translator-2/translator1/variables/variables.data-00000-of-00001
/kaggle/input/final-20-15e/translator/fingerprint.pb
/kaggle/input/final-20-15e/translator/saved_model.pb
/kaggle/input/final-20-15e/translator/variables/variables.index
/k

In [2]:
from nltk.translate.bleu_score import sentence_bleu

In [3]:
!pip install tensorflow-text

In [4]:
import tensorflow as tf
import tensorflow_text as tf_text

In [5]:
!pip install Metaphone
from metaphone import doublemetaphone

In [6]:
import pickle
with open('/kaggle/input/hin-vocab/vocab_data.pkl', 'rb') as fp:
    hin_vocab = pickle.load(fp)
vocab_keys=[l for l in hin_vocab]
len(hin_vocab),len(vocab_keys)

(26214, 26214)

In [7]:
import re
re.sub("[.!?\\-\'\"]", "", "    kaise ho tum viase .?,KSDFaha   ").lower().strip()

'kaise ho tum viase ,ksdfaha'

In [8]:
def t_text(line):
    line=re.sub("[.!?\\-\'\"]", "",line).lower().strip()
    string=''
    for j in line.split(' '):
        if doublemetaphone(j)[0]+'*'+doublemetaphone(j[::-1])[0]+'*'+j[:2]+'*'+j[len(j)-1:] in vocab_keys:
            string=string+list(hin_vocab[doublemetaphone(j)[0]+'*'+doublemetaphone(j[::-1])[0]+'*'+j[:2]+'*'+j[len(j)-1:]])[0]+' '
        else:
            string=string+j+' '
    return string.lower().strip()

In [9]:
import pandas as pd
data_val=pd.read_csv('/kaggle/input/validate-bleu/validation.tsv',sep='\t')
data_val=data_val[['cs_query','en_query']]
data_val=data_val.rename(columns={'cs_query':'Sentence','en_query':'trr'})
data_val['trr']=data_val['trr'].apply(lambda x: x.lower().strip())
data_val

,Sentence,trr
0,mere timer ko roko,pause my timer .
1,Please Liz ke liye video message banaye,please make video message for liz
2,Alarm create karo at 4 pm and 9 pm on tuesday,create an alarm at 4 pm and 9 pm on tuesday
3,Library events Cedar Park me hai,library events in cedar park
4,Please timer ko rokey,please pause timer
...,...,...
1385,Is summer ko outdoor concerts,outdoor concerts this summer
1386,Mai chahta hu ki aap mere alarm ko 10 minutes ...,i ' d like you to snooze my alarm for 10 minut...
1387,please Tupac play kare,please play tupac
1388,Rhonda ko message kare aur July me Vacation Bi...,message rhonda ask her details about vacation ...


In [10]:
'''three_input_text = tf.constant([t_text('sabse best friend'), 
                                t_text('That toh I know'),
                                t_text('She was bhunnoing the masalas jub phone ki ghuntee bugee'),
                                t_text('tum batao'),
                                t_text('kya karna hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhare papa ka naam kya hai'),
                                t_text('tumhara papa ka naam kya hai'),
])

result = reloaded.translate(input_text=three_input_text)

for tr in result['text']:
  print(tr.numpy().decode())'''

"three_input_text = tf.constant([t_text('sabse best friend'), \n                                t_text('That toh I know'),\n                                t_text('She was bhunnoing the masalas jub phone ki ghuntee bugee'),\n                                t_text('tum batao'),\n                                t_text('kya karna hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhare papa ka naam kya hai'),\n                                t_text('tumhara papa ka naam kya hai'),\n])\n\nresult = reloaded.translate(input_text=three_input_text)\n\nfor tr in result['text']:\n  print(tr.numpy().decode())"

In [11]:
import numpy

In [12]:
def cal_bleu(x,y):
    return sentence_bleu(x.split(),y.split())

In [13]:
reloaded = tf.saved_model.load('/kaggle/input/transl-last-hour/translator1')
data_val['model_output1']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score1']=data_val[['trr','model_output1']].apply(lambda x: cal_bleu(*x),axis=1)

/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [14]:
data_val

,Sentence,trr,model_output1,bleu_score1
0,mere timer ko roko,pause my timer .,tell my timer,0.000000
1,Please Liz ke liye video message banaye,please make video message for liz,please give the liz loved cutting a message fr...,0.537285
2,Alarm create karo at 4 pm and 9 pm on tuesday,create an alarm at 4 pm and 9 pm on tuesday,create a reminder at pm and pm on tuesday for ...,0.526640
3,Library events Cedar Park me hai,library events in cedar park,change temperate while there is an kids to get...,0.000000
4,Please timer ko rokey,please pause timer,please pause timer,0.000000
...,...,...,...,...
1385,Is summer ko outdoor concerts,outdoor concerts this summer,outdoor concerts this summer,0.000000
1386,Mai chahta hu ki aap mere alarm ko 10 minutes ...,i ' d like you to snooze my alarm for 10 minut...,"i want you to understand my hands , the cleane...",0.594604
1387,please Tupac play kare,please play tupac,please play tupac,0.000000
1388,Rhonda ko message kare aur July me Vacation Bi...,message rhonda ask her details about vacation ...,message rhonda ask her to ask about vacation b...,0.000000


In [15]:
sentence_bleu("i have eaten".split(), "i have eaten the food".split() )

/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.668740304976422

In [16]:
sentence_bleu("i have eaten the food".split(), "i have eaten".split() )

0.7598356856515925

In [17]:
data_val['bleu_score1'].mean()

0.20418386148695708

In [18]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-015-mixed-00409/translator1')
data_val['model_output2']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score2']=data_val[['trr','model_output2']].apply(lambda x: cal_bleu(*x),axis=1)

/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [19]:
data_val['bleu_score2'].mean()

0.25378788595682233

In [20]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-02-15e/translator1')
data_val['model_output3']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score3']=data_val[['trr','model_output3']].apply(lambda x: cal_bleu(*x),axis=1)

In [21]:
data_val['bleu_score3'].mean()

0.19311232334855913

In [22]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-01-15e/translator1')
data_val['model_output4']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score4']=data_val[['trr','model_output4']].apply(lambda x: cal_bleu(*x),axis=1)

In [23]:
data_val['bleu_score4'].mean()

0.1924679606598974

In [24]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-15e/translator')
data_val['model_output5']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score5']=data_val[['trr','model_output5']].apply(lambda x: cal_bleu(*x),axis=1)

In [25]:
data_val['bleu_score5'].mean()

0.18740454899851572

In [26]:
reloaded = tf.saved_model.load('/kaggle/input/final-20-22e/translator')
data_val['model_output6']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score6']=data_val[['trr','model_output6']].apply(lambda x: cal_bleu(*x),axis=1)

In [27]:
data_val['bleu_score6'].mean()

0.16218631499727978

In [28]:
reloaded = tf.saved_model.load('/kaggle/input/translator1/translator1')
data_val['model_output7']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score7']=data_val[['trr','model_output7']].apply(lambda x: cal_bleu(*x),axis=1)

In [29]:
data_val['bleu_score7'].mean()

0.20382973621876396

In [30]:
reloaded = tf.saved_model.load('/kaggle/input/translator-2/translator1')
data_val['model_output8']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score8']=data_val[['trr','model_output8']].apply(lambda x: cal_bleu(*x),axis=1)

In [31]:
data_val['bleu_score8'].mean()

0.21631971332494507

In [32]:
reloaded = tf.saved_model.load('/kaggle/input/change-vocab-20-01/change_vocab_20_0.1/translator')
reloaded.tf_translate(
                tf.constant([
                t_text('tumhara naam kya hai')
                        ]))['text'][0].numpy().decode()

'hello . . salman khan will have a headache'

In [33]:
reloaded = tf.saved_model.load('/kaggle/input/change-vocab-20-01/change_vocab_20_0.1/translator')
data_val['model_output9']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score9']=data_val[['trr','model_output9']].apply(lambda x: cal_bleu(*x),axis=1)

In [34]:
data_val['bleu_score9'].mean()

0.2739331618925749

In [42]:
reloaded = tf.saved_model.load('/kaggle/input/change-vocab-20-01/change_vocab_20_0.1/translator')

three_input_text = tf.constant([#'sabse best friend', 
                                #'That toh I know',
                                t_text('mera naam rohan hai'),
                                t_text('tum batao'),
                                t_text('kya karna hai'),
])

result = reloaded.tf_translate(three_input_text)
print(result['text'])
for tr in result['text']:
  print(tr.numpy().decode())

tf.Tensor(
[b'my motive to allah loves' b'you made up'
 b'what would have happened . fallen'], shape=(3,), dtype=string)
my motive to allah loves
you made up
what would have happened . fallen


In [35]:
reloaded = tf.saved_model.load('/kaggle/input/file-name-20-02/file_name_20_0.2/translator')
data_val['model_output10']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score10']=data_val[['trr','model_output10']].apply(lambda x: cal_bleu(*x),axis=1)

In [36]:
data_val['bleu_score10'].mean()

0.18979147598171534

In [37]:
reloaded = tf.saved_model.load('/kaggle/input/anuvaad-03/anuvaad_0.3/translator')
data_val['model_output11']=data_val['Sentence'].apply(lambda x: 
                                                    reloaded.tf_translate(
                tf.constant([
                t_text(x)
                        ]))['text'][0].numpy().decode().strip())
data_val['bleu_score11']=data_val[['trr','model_output11']].apply(lambda x: cal_bleu(*x),axis=1)

In [38]:
data_val['bleu_score11'].mean()

0.23638089445476473

In [39]:
data_val

,Sentence,trr,model_output1,bleu_score1,model_output2,bleu_score2,model_output3,bleu_score3,model_output4,bleu_score4,...,model_output7,bleu_score7,model_output8,bleu_score8,model_output9,bleu_score9,model_output10,bleu_score10,model_output11,bleu_score11
0,mere timer ko roko,pause my timer .,tell my timer,0.000000,my timer is completely strong,0.000000,mute my timer,0.000000,pause my timer .,0.550695,...,play my timer,0.000000,,0.000000,first follow your feet,0.000000,stop the fire,0.000000,a friend is pending,0.550695
1,Please Liz ke liye video message banaye,please make video message for liz,please give the liz loved cutting a message fr...,0.537285,"please while the liz , hide the new santiago w...",0.000000,please provide a sixty mr . warning !,0.594604,please set a reminder for the most nba read me...,0.526640,...,please give the liz cocktail lounge made by me...,0.000000,please show the liz dirty bastard of luther na...,0.000000,please suggest the message directly . . the po...,0.000000,please repaid the path of sending the heat,0.000000,that is actual screenshot and i know that bein...,0.526640
2,Alarm create karo at 4 pm and 9 pm on tuesday,create an alarm at 4 pm and 9 pm on tuesday,create a reminder at pm and pm on tuesday for ...,0.526640,amend the reminder for the android association...,0.526640,create an alarm called brother . pm and pm on ...,0.000000,make the celtic party on tuesday . pm and pm o...,0.000000,...,"create to be at pm and pm on tuesday , repeate...",0.000000,create mercy on tuesday at pm and pm on tuesday,0.000000,king rape afternoon pan and group on tuesday !,0.000000,malai ji by pt on the game,0.000000,see the roll number .,0.000000
3,Library events Cedar Park me hai,library events in cedar park,change temperate while there is an kids to get...,0.000000,library events at oct party dam,0.000000,library events locations park tomorrow,0.000000,library events in the library that are oldest ...,0.000000,...,get used to do the library events in cedar breath,0.000000,singing events at cedar straight,0.000000,fist . . dangerous . . . . now put the back of...,0.000000,i want to blow the rumours,0.638943,my plans are far beyond than what you can thin...,0.000000
4,Please timer ko rokey,please pause timer,please pause timer,0.000000,"please pause a timer , how long will be reduced",0.562341,please pause timer,0.000000,please pause timer,0.000000,...,please pause timer,0.000000,please pause timer,0.000000,please suggest the throw . .,0.000000,to blow,0.000000,please send screenshots as today is exam,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1385,Is summer ko outdoor concerts,outdoor concerts this summer,outdoor concerts this summer,0.000000,great summer people ask the tree summer in ord...,0.000000,outdoor concerts this summer,0.000000,pumpkin concerts in the garbage bin,0.000000,...,outdoor concerts this summer,0.000000,outdoor concerts this summer,0.000000,"heat , the toilet of the wind has a voice",0.000000,a summer scoundrel you,0.000000,do it have to study,0.000000
1386,Mai chahta hu ki aap mere alarm ko 10 minutes ...,i ' d like you to snooze my alarm for 10 minut...,"i want you to understand my hands , the cleane...",0.594604,i am going to my father . . . let me sleep . ....,0.555524,i am running you . . . i have only those who h...,0.614788,hey it ! ! allah may you have mercy for my lif...,0.594604,...,i want you to stop my cellphone seven minutes ...,0.604275,i want you to stop my cellphone to minutes pum...,0.594604,may i want ? i have faith in you . . . . as so...,0.543032,"i want to become a bottle , i am very words fo...",0.648139,i had have to type the house for two shop in n...,0.526640
1387,please Tupac play kare,please play tupac,please play tupac,0.000000,please play tupac play selections,0.000000,please play the tupac from the tupac .,0.000000,please play tupac play . no ! !,0.000000,...,please play tupac,0.000000,please play tupac,0.